In [4]:
from pyrosm import OSM, get_data
import osmnx as ox
import networkx as nx
import geopandas as gpd
import requests
import folium

In [5]:
security_by_highway = {
    "cycleway":1,
    "pedestrian":0.9,
    "residential":0.6,
    "living_street":0.6,
    "tertiary":0.7,
    "tertiary_link":0.7,
    "bus_stop":0.7,
    "track":0.6,
    "services":0.7,
    "service":0.7,
    "path":0.6,
    "secondary":0.6,
    "secondary_link":0.6,
    "unclassified":0.5,
    "road":0.5,
    "primary":0.4,
    "primary_link":0.4
}

In [6]:
security_by_cycleway = {
    "lane":1,
    "track":1,
    "separate":1,
    "opposite":0.9,
    "opposite_lane":0.9,
    "opposite_track":0.9,
    "left":0.8,
    "share_busway":0.7,
    "opposite_share_busway":0.6,
    "shared_lane":0.5,
    "yes":0.6,
    "crossing":0.5,
    "none":0,
    "no":0,
    None:0
}

In [7]:
data = OSM( get_data("Bordeaux", directory="data"))

In [8]:
cycling_nodes, cycling_edges = data.get_network(network_type="cycling",nodes=True)

In [9]:
cycling_nodes = cycling_nodes[["id","geometry","lat","lon"]]

In [10]:
cycling_edges = cycling_edges[["id","u","v","geometry","highway","cycleway","surface","bicycle","length","oneway"]]
cycling_edges = cycling_edges[~cycling_edges["highway"].isin(["construction","trunk_link","trunk","footway","steps","bridleway"])]

In [11]:
cycling_edges['security_highway'] = 1 - cycling_edges['highway'].map(security_by_highway)
print(cycling_edges["security_highway"].isna().sum())
cycling_edges["security_highway"] = cycling_edges["security_highway"].fillna(0.5)

0


In [12]:
cycling_edges['security_cycleway'] = 1 - cycling_edges['cycleway'].map(security_by_cycleway)
print(cycling_edges["security_highway"].isna().sum())
cycling_edges["security_cycleway"] = cycling_edges["security_cycleway"].fillna(1)

0


In [13]:
cycling_edges['security_cycleway'].value_counts(dropna=False)

security_cycleway
1.0    450006
0.0      5186
0.1      3100
0.5       536
0.3       298
0.4        66
0.2        15
Name: count, dtype: int64

In [14]:
cycling_edges["score"] = cycling_edges["length"] / (cycling_edges["security_highway"] + cycling_edges["security_cycleway"])

In [15]:
cycling_edges["cycleway"].value_counts(dropna=False)

cycleway
None                     449205
lane                       5020
opposite                   2898
no                          671
shared_lane                 428
share_busway                298
opposite_lane               196
none                        130
track                       126
crossing                    108
opposite_share_busway        58
separate                     40
left                         15
yes                           8
opposite_track                6
Name: count, dtype: int64

In [16]:
graph = data.to_graph(cycling_nodes, cycling_edges, graph_type="networkx")

In [17]:
def get_localisations(address):
    response = requests.get(f"https://api-adresse.data.gouv.fr/search/?q={address.replace(' ','+')}&lat=44.841225&lon=-0.580036&limit=5").json().get('features')
    response = response[0].get('geometry').get('coordinates')
    return response

In [18]:
source_address = "31 rue de peybouquey, Talence, France"
target_address = "Palais de Justice, Bordeaux, France"

In [19]:
source_coordinates = get_localisations(source_address)
target_coordinates = get_localisations(target_address)

In [20]:
print(source_coordinates)
print(target_coordinates)

[-0.601474, 44.817763]
[-0.569327, 44.838414]


In [21]:
print(type(source_coordinates))
print(type(source_coordinates[0]))

<class 'list'>
<class 'float'>


In [22]:
source_node_id = ox.nearest_nodes(graph, source_coordinates[0], source_coordinates[1])
target_node_id = ox.nearest_nodes(graph, target_coordinates[0], target_coordinates[1])

In [46]:
# temporary select random nodes
source_node_id = cycling_nodes.sample(1)["id"].values[0]
target_node_id = cycling_nodes.sample(1)["id"].values[0]

In [23]:
print(source_node_id, target_node_id)

900379563 5878389434


In [24]:
source_node = cycling_nodes[cycling_nodes['id'] == source_node_id]
target_node = cycling_nodes[cycling_nodes['id'] == target_node_id]

In [25]:
print(source_node,target_node)

              id                   geometry       lat       lon
89103  900379563  POINT (-0.60144 44.81765)  44.81765 -0.601439                 id                   geometry        lat      lon
141673  5878389434  POINT (-0.56925 44.83842)  44.838421 -0.56925


In [26]:
route_length = nx.shortest_path(graph,source_node_id,target_node_id,weight='length')
route_security = nx.shortest_path(graph,source_node_id,target_node_id,weight='security')
route_score = nx.shortest_path(graph,source_node_id,target_node_id,weight='score')

In [27]:
print(len(route_length),len(route_security),len(route_score))

230 165 241


In [52]:
import folium
m = folium.Map(location=[44.841225, -0.580036], zoom_start=13)

In [53]:
for i in range(len(route_length)-1) :
    edge = []
    graph_edge = graph.edges[route_length[i],route_length[i+1],0]
    for x,y in graph_edge["geometry"].coords:
        edge.append([y,x])
    tooltip_data = {"cycleway" : graph_edge["cycleway"], "surface" : graph_edge["surface"], "bicycle" : graph_edge["bicycle"], "length" : graph_edge["length"], "oneway" : graph_edge["oneway"], "security_highway" : graph_edge["security_highway"],"security_cycleway": graph_edge["security_cycleway"], "score" : graph_edge["score"], "highway" : graph_edge["highway"]}
    tooltip = folium.Tooltip(tooltip_data)
    folium.PolyLine(locations=edge, color='blue',tooltip=tooltip,opacity=0.7).add_to(m)

In [54]:
for i in range(len(route_security)-1) :
    edge = []
    graph_edge = graph.edges[route_security[i],route_security[i+1],0]
    for x,y in graph_edge["geometry"].coords:
        edge.append([y,x])
    tooltip_data = {"cycleway" : graph_edge["cycleway"], "surface" : graph_edge["surface"], "bicycle" : graph_edge["bicycle"], "length" : graph_edge["length"], "oneway" : graph_edge["oneway"], "security_highway" : graph_edge["security_highway"],"security_cycleway": graph_edge["security_cycleway"], "score" : graph_edge["score"], "highway" : graph_edge["highway"]}
    tooltip = folium.Tooltip(tooltip_data)
    folium.PolyLine(locations=edge, color='yellow',tooltip=tooltip,opacity=0.7).add_to(m)

In [55]:
for i in range(len(route_score)-1) :
    edge = []
    graph_edge = graph.edges[route_score[i],route_score[i+1],0]
    for x,y in graph_edge["geometry"].coords:
        edge.append([y,x])
    tooltip_data = {"cycleway" : graph_edge["cycleway"], "surface" : graph_edge["surface"], "bicycle" : graph_edge["bicycle"], "length" : graph_edge["length"], "oneway" : graph_edge["oneway"], "security_highway" : graph_edge["security_highway"],"security_cycleway": graph_edge["security_cycleway"], "score" : graph_edge["score"], "highway" : graph_edge["highway"]}
    tooltip = folium.Tooltip(tooltip_data)
    folium.PolyLine(locations=edge, color='green',tooltip=tooltip,opacity=0.7).add_to(m)

In [56]:
folium.Marker(location=[source_node.geometry.values[0].y, source_node.geometry.values[0].x], popup="source").add_to(m)
folium.Marker(location=[target_node.geometry.values[0].y, target_node.geometry.values[0].x], popup="target").add_to(m)

In [57]:
# put legend on map
# yellow = security
# green = score
# blue = length
legend_html = """
<div style="position: fixed; 
     bottom: 50px; left: 50px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     ">&nbsp; <b>Legend</b><br>
     &nbsp; Length &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i><br>
     &nbsp; Security &nbsp; <i class="fa fa-map-marker fa-2x" style="color:yellow"></i><br>
     &nbsp; Score &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

In [58]:
m